In [ ]:
import mediapipe as mp 
import cv2
from dollarpy import Recognizer, Template, Point
import os
import numpy as np
import socket
import threading

In [ ]:
templates=[]

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
mp_styles=mp.solutions.drawing_styles

In [ ]:
def getPoints(videoURL, label):
    
    result = None
    cap = cv2.VideoCapture(videoURL)

    with mp.solutions.hands.Hands(max_num_hands=2, min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
        points = []
        wrist = []
        thumb_cmc = []
        thumb_mcp = []
        thumb_ip = []
        thumb_tip = []
        index_finger_mcp = []
        index_finger_pip = []
        index_finger_dip = []
        index_finger_tip = []
        middle_finger_mcp = []
        middle_finger_pip = []
        middle_finger_dip = []
        middle_finger_tip = []
        ring_finger_mcp = []
        ring_finger_pip = []
        ring_finger_dip = []
        ring_finger_tip = []
        pinky_mcp = []
        pinky_pip = []
        pinky_dip = []
        pinky_tip = []

        while cap.isOpened():
            ret, frame = cap.read()
            if ret == True:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image = cv2.flip(image, 1)
                image.flags.writeable = False

                results = hands.process(image)

                image.flags.writeable = True
                if results.multi_hand_landmarks:
                    for hand_landmarks in results.multi_hand_landmarks:
                        mp.solutions.drawing_utils.draw_landmarks(
                        image, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS,
                        mp.solutions.drawing_styles.get_default_hand_landmarks_style(),
                        mp.solutions.drawing_styles.get_default_hand_connections_style()
                        )
                

               
                try:
                    handy = results.multi_hand_landmarks
                    index = 0
                    newlist = []
                    for lnd in handy:
                        if index in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]:
                            newlist.append(lnd)
                        index += 1

                    wrist.append(Point(newlist[0].landmark[0].x, newlist[0].landmark[0].y, 1))
                    thumb_cmc.append(Point(newlist[1].landmark[1].x, newlist[1].landmark[1].y, 2))
                    thumb_mcp.append(Point(newlist[2].landmark[2].x, newlist[2].landmark[2].y, 3))
                    thumb_ip.append(Point(newlist[3].landmark[3].x, newlist[3].landmark[3].y, 4))
                    thumb_tip.append(Point(newlist[4].landmark[4].x, newlist[4].landmark[4].y, 5))
                    index_finger_mcp.append(Point(newlist[5].landmark[5].x, newlist[5].landmark[5].y, 6))
                    index_finger_pip.append(Point(newlist[6].landmark[6].x, newlist[6].landmark[6].y, 7))
                    index_finger_dip.append(Point(newlist[7].landmark[7].x, newlist[7].landmark[7].y, 8))
                    index_finger_tip.append(Point(newlist[8].landmark[8].x, newlist[8].landmark[8].y, 9))
                    middle_finger_mcp.append(Point(newlist[9].landmark[9].x, newlist[9].landmark[9].y, 10))
                    middle_finger_pip.append(Point(newlist[10].landmark[10].x, newlist[10].landmark[10].y, 11))
                    middle_finger_dip.append(Point(newlist[11].landmark[11].x, newlist[11].landmark[11].y, 12))
                    middle_finger_tip.append(Point(newlist[12].landmark[12].x, newlist[12].landmark[12].y, 13))
                    ring_finger_mcp.append(Point(newlist[13].landmark[14].x, newlist[13].landmark[13].y, 14))
                    ring_finger_pip.append(Point(newlist[14].landmark[14].x, newlist[14].landmark[14].y, 15))
                    ring_finger_dip.append(Point(newlist[15].landmark[15].x, newlist[15].landmark[15].y, 16))
                    ring_finger_tip.append(Point(newlist[16].landmark[16].x, newlist[16].landmark[16].y, 17))
                    pinky_mcp.append(Point(newlist[17].landmark[17].x, newlist[17].landmark[17].y, 18))
                    pinky_pip.append(Point(newlist[18].landmark[18].x, newlist[18].landmark[18].y, 19))
                    pinky_dip.append(Point(newlist[19].landmark[19].x, newlist[19].landmark[19].y, 20))
                    pinky_tip.append(Point(newlist[20].landmark[20].x, newlist[20].landmark[20].y, 21))

                except:
                    pass
                
                cv2.imshow(label, image)
                
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    result = "exit"
                    break

    cap.release()
    cv2.destroyAllWindows()
    
    print(label)
    points = wrist + thumb_cmc + thumb_mcp + thumb_ip + thumb_tip + index_finger_mcp + index_finger_pip + \
             index_finger_dip + index_finger_tip + middle_finger_mcp + middle_finger_pip + middle_finger_dip + \
             middle_finger_tip + ring_finger_mcp + ring_finger_pip + ring_finger_dip + ring_finger_tip + pinky_mcp + \
             pinky_pip + pinky_dip + pinky_tip
    total_x = sum(point.x for point in points)
    total_y = sum(point.y for point in points)
    print(f"Total points for {label}: ({total_x}, {total_y})")

    return points

In [ ]:
vid = 0
points = getPoints(vid, "right")
tmpl_2 = Template('right', points)
templates.append(tmpl_2)


vid = 0
points = getPoints(vid,"left") 
tmpl_2 = Template('left', points)
templates.append(tmpl_2)

vid = 0
points = getPoints(vid,"start") 
tmpl_2 = Template('start', points)
templates.append(tmpl_2)

#vid = 0
#points = getPoints(vid,"down") 
#tmpl_2 = Template('down', points)
#templates.append(tmpl_2)

#vid = 0
#points = getPoints(vid,"center") 
#tmpl_2 = Template('center', points)
#templates.append(tmpl_2)



In [ ]:
def recognation(conn):
    global result
    vid = 0
    points = getPoints(vid, "what")
    import time

    start = time.time()
    recognizer = Recognizer(templates)
    result = str(recognizer.recognize(points))
    end = time.time()
    duration = end - start
  


    # Extract the relevant information and remove the first bracket
    result_info = result.split(",")[0][1:]  # Remove the first character (the opening bracket)
    print(result_info)

    # Send only the relevant information
    msg = bytes(result_info, 'utf-8')
    conn.send(msg)

In [ ]:
import socket
import threading

def handle_client(conn, addr):
    print(f"Accepted connection from {addr}")
    
    # Perform any actions you want with the connection here
    # For example, you can call your recognition function
    recognation(conn)
    
    # Close the connection when done
    conn.close()
    print(f"Connection from {addr} closed")

def start_server():
    host = 'localhost'
    port = 3344

    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(5)

    print(f"Server listening on {host}:{port}")

    while True:
        try:
            conn, addr = server_socket.accept()
            client_thread = threading.Thread(target=handle_client, args=(conn, addr))
            client_thread.start()
        except Exception as e:
            print(f"Error accepting connection: {e}")


start_server()